In [1]:
import pickle
import numpy as np
import pandas as pd
import copy
import os
import sys
import time
sys.path.append("/home/ubuntu/CE_scheme/")
from Schemas.imdb.schema import gen_imdb_schema
from Join_scheme.data_prepare import read_table_csv
from Join_scheme.binning import identify_key_values

In [11]:
a = dict()
a[(1)] = 3
a

{1: 3}

In [9]:
sorted(("asd.sad", "sdasew"))

['asd.sad', 'sdasew']

In [12]:
import numpy as np
import copy


class Bucket:
    """
    The class of bucketization of a key attribute
    """

    def __init__(self, name, bins=[], bin_modes=[], bin_vars=[], bin_means=[], rest_bins_remaining=None):
        self.name = name
        self.bins = bins
        self.bin_modes = bin_modes
        self.bin_vars = bin_vars
        self.bin_means = bin_means
        self.rest_bins_remaining = rest_bins_remaining
        if len(bins) != 0:
            assert len(bins) == len(bin_modes)


class Table_bucket:
    """
    The class of bucketization for all key attributes in a table.
    Supporting more than three dimensional bin modes requires simplifying the causal structure
    """
    def __init__(self, table_name, id_attributes, bin_sizes, oned_bin_modes=None):
        self.table_name = table_name
        self.id_attributes = id_attributes
        self.bin_sizes = bin_sizes
        if oned_bin_modes:
            self.oned_bin_modes = oned_bin_modes
        else:
            self.oned_bin_modes = dict()
        self.twod_bin_modes = dict()


class Bucket_group:
    """
    The class of bucketization for a group of equivalent join keys
    """

    def __init__(self, buckets, start_key, sample_rate, bins=None, primary_keys=[]):
        self.buckets = buckets
        self.start_key = start_key
        self.sample_rate = sample_rate
        self.bins = bins
        self.primary_keys = primary_keys

    def bucketize(self, data):
        """
        Discretize data based on the bucket
        """
        res = dict()
        seen_remain_key = np.array([])
        cumulative_bin = copy.deepcopy(self.buckets[self.start_key].bins)
        start_means = np.asarray(self.buckets[self.start_key].bin_means)

        for key in data:
            if key in self.primary_keys:
                continue
            res[key] = copy.deepcopy(data[key])
            if key != self.start_key:
                unique_remain = np.setdiff1d(self.buckets[key].rest_bins_remaining, seen_remain_key)
                assert sum([np.sum(np.isin(unique_remain, b) == 1) for b in cumulative_bin]) == 0

                if len(unique_remain) != 0:
                    remaining_data = data[key][np.isin(data[key], unique_remain)]
                    unique_remain, count_remain = np.unique(remaining_data, return_counts=True)
                    unique_counts = np.unique(count_remain)
                    for u in unique_counts:
                        temp_idx = np.searchsorted(start_means, u)
                        if temp_idx == len(cumulative_bin):
                            idx = -1
                            if u > self.buckets[key].bin_modes[-1]:
                                self.buckets[key].bin_modes[-1] = u
                        elif temp_idx == 0:
                            idx = 0
                        else:
                            if (u - start_means[temp_idx - 1]) >= (start_means[temp_idx] - u):
                                idx = temp_idx - 1
                            else:
                                idx = temp_idx
                        temp_unique = unique_remain[count_remain == u]
                        cumulative_bin[idx] = np.concatenate((cumulative_bin[idx], temp_unique))
                        seen_remain_key = np.concatenate((seen_remain_key, temp_unique))
                        if u > self.buckets[key].bin_modes[idx]:
                            self.buckets[key].bin_modes[idx] = u
            res[key] = copy.deepcopy(data[key])
            count = 0
            for i, b in enumerate(cumulative_bin):
                count += len(data[key][np.isin(data[key], b)])
                res[key][np.isin(data[key], b)] = i

        self.bins = cumulative_bin

        for key in data:
            if key in self.primary_keys:
                res[key] = self.bucketize_PK(data[key])
                self.buckets[key] = Bucket(key, bin_modes=np.ones(len(self.bins)))
        
        for key in data:
            if key in self.primary_keys:
                continue
            if self.sample_rate[key] < 1.0:
                bin_modes = np.asarray(self.buckets[key].bin_modes)
                bin_modes[bin_modes != 1] = bin_modes[bin_modes != 1] / self.sample_rate[key]
                self.buckets[key].bin_modes = bin_modes
        
        return res

    def bucketize_PK(self, data):
        res = copy.deepcopy(data)
        remaining_data = np.unique(data)
        for i, b in enumerate(self.bins):
            res[np.isin(data, b)] = i
            remaining_data = np.setdiff1d(remaining_data, b)
        if len(remaining_data) != 0:
            self.bins.append(list(remaining_data))
            for key in self.buckets:
                if key not in self.primary_keys:
                    self.buckets[key].bin_modes = np.append(self.buckets[key].bin_modes, 0)
        res[np.isin(data, remaining_data)] = len(self.bins)
        return res



def identify_key_values(schema):
    """
    identify all the key attributes from the schema of a DB, currently we assume all possible joins are known
    It is also easy to support unseen joins, which we left as a future work.
    :param schema: the schema of a DB
    :return: a dict of all keys, {table: [keys]};
             a dict of set, each indicating which keys on different tables are considered the same key.
    """
    all_keys = set()
    equivalent_keys = dict()
    for i, join in enumerate(schema.relationships):
        keys = join.identifier.split(" = ")
        all_keys.add(keys[0])
        all_keys.add(keys[1])
        seen = False
        for k in equivalent_keys:
            if keys[0] in equivalent_keys[k]:
                equivalent_keys[k].add(keys[1])
                seen = True
                break
            elif keys[1] in equivalent_keys[k]:
                equivalent_keys[k].add(keys[0])
                seen = True
                break
        if not seen:
            # set the keys[-1] as the identifier of this equivalent join key group for convenience.
            equivalent_keys[keys[-1]] = set(keys)

    assert len(all_keys) == sum([len(equivalent_keys[k]) for k in equivalent_keys])
    return all_keys, equivalent_keys


def equal_freq_binning(name, data, n_bins, data_len):
    uniques, counts = data
    if len(uniques) <= n_bins:
        bins = []
        bin_modes = []
        bin_vars = []
        bin_means = []
        
        for i, uni in enumerate(uniques):
            bins.append([uni])
            bin_modes.append(counts[i])
            bin_vars.append(0)
            bin_means.append(counts[i])
        return Bucket(name, bins, bin_modes, bin_vars, bin_means)
    
    unique_counts, count_counts = np.unique(counts, return_counts=True)
    idx = np.argsort(unique_counts)
    unique_counts = unique_counts[idx]
    count_counts = count_counts[idx]

    bins = []
    bin_modes = []
    bin_vars = []
    bin_means = []

    bin_freq = data_len / n_bins
    cur_freq = 0
    cur_bin = []
    cur_bin_count = []
    for i, uni_c in enumerate(unique_counts):
        cur_freq += count_counts[i] * uni_c
        cur_bin.append(uniques[np.where(counts == uni_c)[0]])
        cur_bin_count.extend([uni_c] * count_counts[i])
        if (cur_freq > bin_freq) or (i == (len(unique_counts) - 1)):
            bins.append(np.concatenate(cur_bin))
            cur_bin_count = np.asarray(cur_bin_count)
            bin_modes.append(uni_c)
            bin_means.append(np.mean(cur_bin_count))
            bin_vars.append(np.var(cur_bin_count))
            cur_freq = 0
            cur_bin = []
            cur_bin_count = []
    assert len(uniques) == sum([len(b) for b in bins]), f"some unique values missed or duplicated"
    return Bucket(name, bins, bin_modes, bin_vars, bin_means)


def compute_variance_score(buckets):
    """
    compute the variance of products of random variables
    """
    all_mean = np.asarray([buckets[k].bin_means for k in buckets])
    all_var = np.asarray([buckets[k].bin_vars for k in buckets])
    return np.sum(np.prod(all_var + all_mean ** 2, axis=0) - np.prod(all_mean, axis=0) ** 2)


def sub_optimal_bucketize(data, sample_rate, n_bins=30, primary_keys=[]):
    """
    Perform sub-optimal bucketization on a group of equivalent join keys.
    :param data: a dict of (potentially sampled) table data of the keys
                 the keys of this dict are one group of equivalent join keys
    :param sample_rate: the sampling rate the data, could be all 1 if no sampling is performed
    :param n_bins: how many bins can we allocate
    :param primary_keys: the primary keys in the equivalent group since we don't need to bucketize PK.
    :return: new data, where the keys are bucketized
             the mode of each bucket
    """
    unique_values = dict()
    for key in data:
        if key not in primary_keys:
            unique_values[key] = np.unique(data[key], return_counts=True)

    best_variance_score = np.infty
    best_bin_len = 0
    best_start_key = None
    best_buckets = None
    for start_key in data:
        if start_key in primary_keys:
            continue
        start_bucket = equal_freq_binning(start_key, unique_values[start_key], n_bins, len(data[start_key]))
        rest_buckets = dict()
        for key in data:
            if key == start_key or key in primary_keys:
                continue
            uniques = unique_values[key][0]
            counts = unique_values[key][1]
            rest_buckets[key] = Bucket(key, [], [0] * len(start_bucket.bins), [0] * len(start_bucket.bins),
                                       [0] * len(start_bucket.bins), uniques)
            for i, bin in enumerate(start_bucket.bins):
                idx = np.where(np.isin(uniques, bin) == 1)[0]
                if len(idx) != 0:
                    bin_count = counts[idx]
                    unique_bin_keys = uniques[idx]
                    # unique_bin_count = np.unique(bin_count)
                    # bin_count = np.concatenate([counts[counts == j] for j in unique_bin_count])
                    # unique_bin_keys = np.concatenate([uniques[counts == j] for j in unique_bin_count])
                    rest_buckets[key].rest_bins_remaining = np.setdiff1d(rest_buckets[key].rest_bins_remaining,
                                                                         unique_bin_keys)
                    rest_buckets[key].bin_modes[i] = np.max(bin_count)
                    rest_buckets[key].bin_vars[i] = np.var(bin_count)
                    rest_buckets[key].bin_means[i] = np.mean(bin_count)

        rest_buckets[start_key] = start_bucket
        var_score = compute_variance_score(rest_buckets)
        if len(start_bucket.bins) > best_bin_len:
            best_variance_score = var_score
            best_start_key = start_key
            best_buckets = rest_buckets
            best_bin_len = len(start_bucket.bins)
        elif len(start_bucket.bins) >= best_bin_len * 0.9 and var_score < best_variance_score:
            best_variance_score = var_score
            best_start_key = start_key
            best_buckets = rest_buckets
            best_bin_len = len(start_bucket.bins)
    
    best_buckets = Bucket_group(best_buckets, best_start_key, sample_rate, primary_keys=primary_keys)
    new_data = best_buckets.bucketize(data)
    return new_data, best_buckets


def fixed_start_key_bucketize(start_key, data, sample_rate, n_bins=30, primary_keys=[]):
    """
    Perform sub-optimal bucketization on a group of equivalent join keys based on the pre-defined start_key.
    :param data: a dict of (potentially sampled) table data of the keys
                 the keys of this dict are one group of equivalent join keys
    :param sample_rate: the sampling rate the data, could be all 1 if no sampling is performed
    :param n_bins: how many bins can we allocate
    :param primary_keys: the primary keys in the equivalent group since we don't need to bucketize PK.
    :return: new data, where the keys are bucketized
             the mode of each bucket
    """
    unique_values = dict()
    for key in data:
        if key not in primary_keys:
            unique_values[key] = np.unique(data[key], return_counts=True)

    start_bucket = equal_freq_binning(start_key, unique_values[start_key], n_bins, len(data[start_key]))
    rest_buckets = dict()
    for key in data:
        if key == start_key or key in primary_keys:
            continue
        uniques = unique_values[key][0]
        counts = unique_values[key][1]
        rest_buckets[key] = Bucket(key, [], [0] * len(start_bucket.bins), [0] * len(start_bucket.bins),
                                   [0] * len(start_bucket.bins), uniques)
        for i, bin in enumerate(start_bucket.bins):
            idx = np.where(np.isin(uniques, bin) == 1)[0]
            if len(idx) != 0:
                bin_count = counts[idx]
                unique_bin_keys = uniques[idx]
                rest_buckets[key].rest_bins_remaining = np.setdiff1d(rest_buckets[key].rest_bins_remaining,
                                                                     unique_bin_keys)
                rest_buckets[key].bin_modes[i] = np.max(bin_count)
                rest_buckets[key].bin_means[i] = np.mean(bin_count)

    best_buckets = Bucket_group(rest_buckets, start_key, sample_rate, primary_keys=primary_keys)
    new_data = best_buckets.bucketize(data)
    return new_data, best_buckets



In [13]:
data_path = "/home/ubuntu/data_CE/{}.csv"
schema = gen_imdb_schema(data_path)
all_keys, equivalent_keys = identify_key_values(schema)
with open("new_table_buckets.pkl", "rb") as f:
    table_buckets = pickle.load(f)

In [14]:
with open("/home/ubuntu/data_CE/job/job_sub_plan_queries.txt", "r") as f:
    sub_plan_queries = f.read()
psql_raw = sub_plan_queries.split("query: 0")[1:]
queries = []
q_file_names = []
query_path = "/home/ubuntu/data_CE/job/"
for file in os.listdir(query_path):
    if file.endswith(".sql") and file[0].isnumeric():
        q_file_names.append(file.split(".sql")[0]+".pkl")
        with open(query_path+file, "r") as f:
            q = f.readline()
            queries.append(q)

In [15]:
psql_raw = sub_plan_queries.split("query: 0")[1:]
sub_plan_queries_str_all = []
for per_query in psql_raw:
    sub_plan_queries = []
    sub_plan_queries_str = []
    num_sub_plan_queries = len(per_query.split("query: "))
    all_info = per_query.split("RELOPTINFO (")[1:]
    assert num_sub_plan_queries*2 == len(all_info)
    for i in range(num_sub_plan_queries):
        idx = i*2
        table1 = all_info[idx].split("): rows=")[0]
        table2 = all_info[idx+1].split("): rows=")[0]
        table_str = (table1, table2)
        sub_plan_queries_str.append(table_str)
    sub_plan_queries_str_all.append(sub_plan_queries_str)

In [16]:
import numpy as np
import copy

from Join_scheme.join_graph import get_join_hyper_graph, parse_query_all_join
from Join_scheme.data_prepare import identify_key_values
#from Sampling.load_sample import load_sample_imdb_one_query


class Factor:
    """
    This the class defines a multidimensional conditional probability on one table.
    """
    def __init__(self, table, table_len, variables, pdfs, na_values=None):
        self.table = table
        self.table_len = table_len
        self.variables = variables
        self.pdfs = pdfs
        self.na_values = na_values  # this is the percentage of data, which is not nan.


class Group_Factor:
    """
        This the class defines a multidimensional conditional probability on a group of tables.
    """
    def __init__(self, tables, tables_size, pdfs, bin_modes, equivalent_groups=None, table_key_equivalent_group=None,
                 na_values=None, join_cond=None):
        self.table = tables
        self.tables_size = tables_size
        self.pdfs = pdfs
        self.bin_modes = bin_modes
        self.equivalent_groups = equivalent_groups
        self.table_key_equivalent_group = table_key_equivalent_group
        self.na_values = na_values
        self.join_cond = join_cond


class Bound_ensemble:
    """
    This the class where we store all the trained models and perform inference on the bound.
    """
    def __init__(self, table_buckets, schema, ground_truth_factors_no_filter=None):
        self.table_buckets = table_buckets
        self.schema = schema
        self.all_keys, self.equivalent_keys = identify_key_values(schema)
        self.all_join_conds = None
        self.ground_truth_factors_no_filter = ground_truth_factors_no_filter
        #self.reverse_table_alias = None

    def parse_query_simple(self, query):
        """
        If your selection query contains no aggregation and nested sub-queries, you can use this function to parse a
        join query. Otherwise, use parse_query function.
        """
        tables_all, join_cond, join_keys = parse_query_all_join(query)
        #TODO: implement functions on parsing filter conditions.
        table_filters = dict()
        return tables_all, table_filters, join_cond, join_keys

    def get_all_id_conidtional_distribution(self, query_file_name, tables_alias, join_keys):
        #TODO: make it work on query-driven and sampling based
        return load_sample_imdb_one_query(self.table_buckets, tables_alias, query_file_name, join_keys,
                                          self.ground_truth_factors_no_filter)


    def eliminate_one_key_group(self, tables, key_group, factors, relevant_keys):
        """This version only supports 2D distributions (i.e. the distribution learned with tree-structured PGM)"""
        rest_group = None
        rest_group_cardinalty = 0
        eliminated_tables = []
        rest_group_tables = []
        for table in tables:
            assert key_group in factors[table].equivalent_variables
            temp = copy.deepcopy(factors[table].equivalent_variables)
            temp.remove(key_group)
            if len(temp) == 0:
                eliminated_tables.append(table)
            for key in temp:
                if rest_group:
                    assert factors[table].cardinalities[key] == rest_group_cardinalty
                    rest_group_tables.append(table)
                else:
                    rest_group = key
                    rest_group_cardinalty = factors[table].cardinalities[key]
                    rest_group_tables = [table]

        all_probs_eliminated = []
        all_modes_eliminated = []
        for table in eliminated_tables:
            bin_modes = self.table_buckets[table].oned_bin_modes[relevant_keys[key_group][table]]
            all_probs_eliminated.append(factors[table].pdfs)
            all_modes_eliminated.append(np.minimum(bin_modes, factors[table].pdfs))

        if rest_group:
            new_factor_pdf = np.zeros(rest_group_cardinalty)
        else:
            return self.compute_bound_oned(all_probs_eliminated, all_modes_eliminated)

        for i in range(rest_group_cardinalty):
            for table in rest_group_tables:
                idx_f = factors[table].equivalent_variables.index(key_group)
                idx_b = self.table_buckets[table].id_attributes.index(relevant_keys[key_group][table])
                bin_modes = self.table_buckets[table].twod_bin_modes[relevant_keys[key_group][table]]
                if idx_f == 0 and idx_b == 0:
                    all_probs_eliminated.append(factors[table].pdfs[:, i])
                    all_modes_eliminated.append(np.minimum(bin_modes[:, i], factors[table].pdfs[:, i]))
                elif idx_f == 0 and idx_b == 1:
                    all_probs_eliminated.append(factors[table].pdfs[:, i])
                    all_modes_eliminated.append(np.minimum(bin_modes[i, :], factors[table].pdfs[:, i]))
                elif idx_f == 1 and idx_b == 0:
                    all_probs_eliminated.append(factors[table].pdfs[i, :])
                    all_modes_eliminated.append(np.minimum(bin_modes[:, i], factors[table].pdfs[i, :]))
                else:
                    all_probs_eliminated.append(factors[table].pdfs[i, :])
                    all_modes_eliminated.append(np.minimum(bin_modes[i, :], factors[table].pdfs[i, :]))
            new_factor_pdf[i] = self.compute_bound_oned(all_probs_eliminated, all_modes_eliminated)

        for table in rest_group_tables:
            factors[table] = Factor([rest_group], new_factor_pdf, [rest_group])

        return None

    def compute_bound_oned(self, all_probs, all_modes, return_factor=False):
        temp_all_modes = []
        for i in range(len(all_modes)):
            temp_all_modes.append(np.minimum(all_probs[i], all_modes[i]))
        all_probs = np.stack(all_probs, axis=0)
        temp_all_modes = np.stack(temp_all_modes, axis=0)
        multiplier = np.prod(temp_all_modes, axis=0)
        non_zero_idx = np.where(multiplier != 0)[0]
        min_number = np.amin(all_probs[:, non_zero_idx]/temp_all_modes[:, non_zero_idx], axis=0)
        #print(min_number, multiplier[non_zero_idx])
        if return_factor:
            new_probs = np.zeros(multiplier.shape)
            new_probs[non_zero_idx] = multiplier[non_zero_idx] * min_number
            return new_probs, multiplier
        else:
            multiplier[non_zero_idx] = multiplier[non_zero_idx] * min_number
            return np.sum(multiplier)

    def get_optimal_elimination_order(self, equivalent_group, join_keys, factors):
        """
        This function determines the optimial elimination order for each key group
        """
        cardinalities = dict()
        lengths = dict()
        tables_involved = dict()
        relevant_keys = dict()
        for group in equivalent_group:
            relevant_keys[group] = dict()
            lengths[group] = len(equivalent_group[group])
            cardinalities[group] = []
            tables_involved[group] = set([])
            for keys in equivalent_group[group]:
                for table in join_keys:
                    if keys in join_keys[table]:
                        cardinalities[group].append(len(join_keys[table]))
                        tables_involved[group].add(table)
                        variables = factors[table].variables
                        variables[variables.index(keys)] = group
                        factors[table].variables = variables
                        relevant_keys[group][table] = keys
                        break
            cardinalities[group] = np.asarray(cardinalities[group])

        optimal_order = list(equivalent_group.keys())
        for i in range(len(optimal_order)):
            min_idx = i
            for j in range(i+1, len(optimal_order)):
                min_group = optimal_order[min_idx]
                curr_group = optimal_order[j]
                if np.max(cardinalities[curr_group]) < np.max(cardinalities[min_group]):
                    min_idx = j
                else:
                    min_max_tables = np.max(cardinalities[min_group])
                    min_num_max_tables = len(np.where(cardinalities[min_group] == min_max_tables)[0])
                    curr_max_tables = np.max(cardinalities[curr_group])
                    curr_num_max_tables = len(np.where(cardinalities[curr_group] == curr_max_tables)[0])
                    if curr_num_max_tables < min_num_max_tables:
                        min_idx = j
                    elif lengths[curr_group] < lengths[min_group]:
                        min_idx = j
            optimal_order[i], optimal_order[min_idx] = optimal_order[min_idx], optimal_order[i]
        return optimal_order, tables_involved, relevant_keys

    def get_cardinality_bound_one(self, query_str):
        tables_all, table_queries, join_cond, join_keys = self.parse_query_simple(query_str)
        equivalent_group = get_join_hyper_graph(join_keys, self.equivalent_keys)
        conditional_factors = self.get_all_id_conidtional_distribution(table_queries, join_keys, equivalent_group)
        optimal_order, tables_involved, relevant_keys = self.get_optimal_elimination_order(equivalent_group, join_keys,
                                                                            conditional_factors)

        for key_group in optimal_order:
            tables = tables_involved[key_group]
            res = self.eliminate_one_key_group(tables, key_group, conditional_factors, relevant_keys)
        return res

    def get_sub_plan_queries_sql(self, query_str, sub_plan_query_str_all, query_name=None):
        tables_all, table_queries, join_cond, join_keys = self.parse_query_simple(query_str)
        equivalent_group, table_equivalent_group, table_key_equivalent_group = get_join_hyper_graph(join_keys,
                                                                                            self.equivalent_keys)
        cached_sub_queries_sql = dict()
        cached_union_key_group = dict()
        res_sql = []
        for (left_tables, right_tables) in sub_plan_query_str_all:
            assert " " not in left_tables, f"{left_tables} contains more than one tables, violating left deep plan"
            sub_plan_query_list = right_tables.split(" ") + [left_tables]
            sub_plan_query_list.sort()
            sub_plan_query_str = " ".join(sub_plan_query_list)  # get the string name of the sub plan query
            sql_header = "SELECT COUNT(*) FROM "
            for alias in sub_plan_query_list:
                sql_header += (tables_all[alias] + " AS " + alias + ", ")
            sql_header = sql_header[:-2] + " WHERE "
            if " " in right_tables:
                assert right_tables in cached_sub_queries_sql, f"{right_tables} not in cache, input is not ordered"
                right_sql = cached_sub_queries_sql[right_tables]
                right_union_key_group = cached_union_key_group[right_tables]
                if left_tables in table_queries:
                    left_sql = table_queries[left_tables]
                    curr_sql = right_sql + " AND (" + left_sql + ")"
                else:
                    curr_sql = right_sql
                additional_joins, union_key_group = self.get_additional_join_with_table_group(left_tables,
                                                                        right_union_key_group,
                                                                        table_equivalent_group,
                                                                        table_key_equivalent_group)
                for join in additional_joins:
                    curr_sql = curr_sql + " AND " + join
            else:
                curr_sql = ""
                if left_tables in table_queries:
                    curr_sql += ("(" + table_queries[left_tables] + ")")
                if right_tables in table_queries:
                    if curr_sql != "":
                        curr_sql += " AND "
                    curr_sql += ("(" + table_queries[right_tables] + ")")

                additional_joins, union_key_group = self.get_additional_joins_two_tables(left_tables, right_tables,
                                                                        table_equivalent_group,
                                                                        table_key_equivalent_group)
                for join in additional_joins:
                    if curr_sql == "":
                        curr_sql += join
                    else:
                        curr_sql = curr_sql + " AND " + join
            cached_sub_queries_sql[sub_plan_query_str] = curr_sql
            cached_union_key_group[sub_plan_query_str] = union_key_group
            res_sql.append(sql_header + curr_sql + ";")
        return res_sql

    def get_cardinality_bound_all(self, query_str, sub_plan_query_str_all, query_name=None, debug=False, true_card=None):
        """
        Get the cardinality bounds for all sub_plan_queires of a query.
        Note: Due to efficiency, this current version only support left_deep plans (like the one generated by postgres),
              but it can easily support right deep or bushy plans.
        :param query_str: the target query
        :param sub_plan_query_str_all: all sub_plan_queries of the target query,
               it should be sorted by number of the tables in the sub_plan_query
        """
        tables_all, table_queries, join_cond, join_keys = self.parse_query_simple(query_str)
        #print(join_cond)
        #print(join_keys)
        equivalent_group, table_equivalent_group, table_key_equivalent_group, table_key_group_map = \
            get_join_hyper_graph(join_keys, self.equivalent_keys)
        conditional_factors = self.get_all_id_conidtional_distribution(query_name, tables_all, join_keys)
        #self.reverse_table_alias = {v: k for k, v in tables_all.items()}
        cached_sub_queries = dict()
        cardinality_bounds = []
        for i, (left_tables, right_tables) in enumerate(sub_plan_query_str_all):
            assert " " not in left_tables, f"{left_tables} contains more than one tables, violating left deep plan"
            sub_plan_query_list = right_tables.split(" ") + [left_tables]
            sub_plan_query_list.sort()
            sub_plan_query_str = " ".join(sub_plan_query_list)  #get the string name of the sub plan query
            #print(sub_plan_query_str)
            if " " in right_tables:
                assert right_tables in cached_sub_queries, f"{right_tables} not in cache, input is not ordered"
                right_bound_factor = cached_sub_queries[right_tables]
                curr_bound_factor, res = self.join_with_one_table(sub_plan_query_str,
                                                                  left_tables,
                                                                  tables_all,
                                                                  right_bound_factor,
                                                                  conditional_factors[left_tables],
                                                                  table_equivalent_group,
                                                                  table_key_equivalent_group,
                                                                  table_key_group_map,
                                                                  join_cond)
            else:
                curr_bound_factor, res = self.join_two_tables(sub_plan_query_str,
                                                              left_tables,
                                                              right_tables,
                                                              tables_all,
                                                              conditional_factors,
                                                              join_keys,
                                                              table_equivalent_group,
                                                              table_key_equivalent_group,
                                                              table_key_group_map,
                                                              join_cond)
            cached_sub_queries[sub_plan_query_str] = curr_bound_factor
            res = max(res, 1)
            if debug:
                if true_card[i] == -1:
                    error = "NA"
                else:
                    error = max(res/true_card[i], true_card[i]/res)
                print(f"{left_tables}, {right_tables}|| estimate: {res}, true: {true_card[i]}, error: {error}")
            cardinality_bounds.append(res)
        return cardinality_bounds


    def join_with_one_table(self, sub_plan_query_str, left_table, tables_all, right_bound_factor, cond_factor_left,
                            table_equivalent_group, table_key_equivalent_group, table_key_group_map, join_cond):
        """
        Get the cardinality bound by joining the left_table with the seen right_tables
        :param left_table:
        :param right_tables:
        """
        equivalent_key_group, union_key_group_set, union_key_group, new_join_cond = \
            self.get_join_keys_with_table_group(left_table, right_bound_factor, tables_all, table_equivalent_group,
                                                table_key_equivalent_group, table_key_group_map, join_cond)
        bin_mode_left = self.table_buckets[tables_all[left_table]].oned_bin_modes
        bin_mode_right = right_bound_factor.bin_modes
        key_group_pdf = dict()
        key_group_bin_mode = dict()
        new_union_key_group = dict()
        new_na_values = dict()
        res = right_bound_factor.tables_size
        print("==================================================")
        for key_group in equivalent_key_group:
            print(key_group, equivalent_key_group[key_group], res)
            #print(cond_factor_left.na_values)
            #print(right_bound_factor.na_values)
            all_pdfs = [cond_factor_left.pdfs[key] * cond_factor_left.table_len * cond_factor_left.na_values[key]
                        for key in equivalent_key_group[key_group]["left"]] + \
                       [right_bound_factor.pdfs[key] * res * right_bound_factor.na_values[key]
                        for key in equivalent_key_group[key_group]["right"]]
            all_bin_modes = [bin_mode_left[key] for key in equivalent_key_group[key_group]["left"]] + \
                            [bin_mode_right[key] for key in equivalent_key_group[key_group]["right"]]
            if key_group == "info_type.id":
                print(all_pdfs)
                print("****************************************************")
                print(all_bin_modes)
            new_pdf, new_bin_mode = self.compute_bound_oned(all_pdfs, all_bin_modes, return_factor=True)
            res = np.sum(new_pdf)
            if res == 0:
                res = 10.0
                new_pdf[-1] = 1
                key_group_pdf[key_group] = new_pdf
            else:
                key_group_pdf[key_group] = new_pdf / res
            key_group_bin_mode[key_group] = new_bin_mode
            new_union_key_group[key_group] = [key_group]
            new_na_values[key_group] = 1

        for group in union_key_group:
            table, keys = union_key_group[group]
            new_union_key_group[group] = keys
            for key in keys:
                if table == "left":
                    key_group_pdf[key] = cond_factor_left.pdfs[key]
                    key_group_bin_mode[key] = self.table_buckets[tables_all[left_table]].oned_bin_modes[key]
                    new_na_values[key] = cond_factor_left.na_values[key]
                else:
                    key_group_pdf[key] = right_bound_factor.pdfs[key]
                    key_group_bin_mode[key] = right_bound_factor.bin_modes[key]
                    new_na_values[key] = right_bound_factor.na_values[key]

        new_factor = Group_Factor(sub_plan_query_str, res, key_group_pdf, key_group_bin_mode,
                                  union_key_group_set, new_union_key_group, new_na_values)
        return new_factor, res

    def get_join_keys_with_table_group(self, left_table, right_bound_factor, tables_all, table_equivalent_group,
                                       table_key_equivalent_group, table_key_group_map, join_cond):
        """
            Get the join keys between two tables
        """

        actual_join_cond = []
        for cond in join_cond[left_table]:
            if cond in right_bound_factor.join_cond:
                actual_join_cond.append(cond)
        equivalent_key_group = dict()
        union_key_group_set = table_equivalent_group[left_table].union(right_bound_factor.equivalent_groups)
        union_key_group = dict()
        new_join_cond = right_bound_factor.join_cond.union(join_cond[left_table])
        if len(actual_join_cond) != 0:
            for cond in actual_join_cond:
                key1 = cond.split("=")[0].strip()
                key2 = cond.split("=")[1].strip()
                if key1.split(".")[0] == left_table:
                    key_left = key1.replace(left_table, tables_all[left_table])
                    key_group = table_key_group_map[left_table][key_left]
                    if key_group not in equivalent_key_group:
                        equivalent_key_group[key_group] = dict()
                    if left_table in equivalent_key_group[key_group]:
                        equivalent_key_group[key_group]["left"].append(key_left)
                    else:
                        equivalent_key_group[key_group]["left"] = [key_left]
                    right_table = key2.split(".")[0]
                    key_right = key2.replace(right_table, tables_all[right_table])
                    key_group_t = table_key_group_map[right_table][key_right]
                    assert key_group_t == key_group, f"key group mismatch for join {cond}"
                    if "right" in equivalent_key_group[key_group]:
                        equivalent_key_group[key_group]["right"].append(key_right)
                    else:
                        equivalent_key_group[key_group]["right"] = [key_right]
                else:
                    assert key2.split(".")[0] == left_table, f"unrecognized table alias"
                    key_left = key2.replace(left_table, tables_all[left_table])
                    key_group = table_key_group_map[left_table][key_left]
                    if key_group not in equivalent_key_group:
                        equivalent_key_group[key_group] = dict()
                    if left_table in equivalent_key_group[key_group]:
                        equivalent_key_group[key_group]["left"].append(key_left)
                    else:
                        equivalent_key_group[key_group]["left"] = [key_left]
                    right_table = key1.split(".")[0]
                    key_right = key1.replace(right_table, tables_all[right_table])
                    key_group_t = table_key_group_map[right_table][key_right]
                    assert key_group_t == key_group, f"key group mismatch for join {cond}"
                    if "right" in equivalent_key_group[key_group]:
                        equivalent_key_group[key_group]["right"].append(key_right)
                    else:
                        equivalent_key_group[key_group]["right"] = [key_right]

            for group in union_key_group_set:
                if group in equivalent_key_group:
                    continue
                elif group in table_key_equivalent_group[left_table]:
                    union_key_group[group] = ("left", table_key_equivalent_group[left_table][group])
                else:
                    union_key_group[group] = ("right", right_bound_factor.table_key_equivalent_group[group])

        else:
            common_key_group = table_equivalent_group[left_table].intersection(right_bound_factor.equivalent_groups)
            common_key_group = list(common_key_group)[0]
            for group in union_key_group_set:
                if group == common_key_group:
                    equivalent_key_group[group] = dict()
                    equivalent_key_group[group]["left"] = table_key_equivalent_group[left_table][group]
                    equivalent_key_group[group]["right"] = right_bound_factor.table_key_equivalent_group[group]
                elif group in table_key_equivalent_group[left_table]:
                    union_key_group[group] = ("left", table_key_equivalent_group[left_table][group])
                else:
                    union_key_group[group] = ("right", right_bound_factor.table_key_equivalent_group[group])

        return equivalent_key_group, union_key_group_set, union_key_group, new_join_cond

    def get_additional_join_with_table_group(self, left_table, right_union_key_group, table_equivalent_group,
                                             table_key_equivalent_group):
        common_key_group = table_equivalent_group[left_table].intersection(set(right_union_key_group.keys()))
        union_key_group_set = table_equivalent_group[left_table].union(set(right_union_key_group.keys()))
        union_key_group = copy.deepcopy(right_union_key_group)
        all_join_predicates = []
        for group in union_key_group_set:
            if group in common_key_group:
                left_key = table_key_equivalent_group[left_table][group][0]
                left_key = left_table + "." + left_key.split(".")[-1]
                right_key = right_union_key_group[group]
                join_predicate = left_key + " = " + right_key
                all_join_predicates.append(join_predicate)
            if group not in union_key_group:
                assert group in table_key_equivalent_group[left_table]
                left_key = table_key_equivalent_group[left_table][group][0]
                left_key = left_table + "." + left_key.split(".")[-1]
                union_key_group[group] = left_key

        return all_join_predicates, union_key_group

    def join_two_tables(self, sub_plan_query_str, left_table, right_table, tables_all, conditional_factors, join_keys,
                        table_equivalent_group, table_key_equivalent_group, table_key_group_map, join_cond):
        """
            Get the cardinality bound by joining the left_table with the right_table
            :param left_table:
            :param right_table:
        """
        equivalent_key_group, union_key_group_set, union_key_group, new_join_cond = \
            self.get_join_keys_two_tables(left_table, right_table, table_equivalent_group, table_key_equivalent_group,
                                          table_key_group_map, join_cond, join_keys, tables_all)
        #print(left_table, right_table)
        #print(equivalent_key_group)
        #print(union_key_group)
        #print(conditional_factors.keys())
        cond_factor_left = conditional_factors[left_table]
        cond_factor_right = conditional_factors[right_table]
        bin_mode_left = self.table_buckets[tables_all[left_table]].oned_bin_modes
        bin_mode_right = self.table_buckets[tables_all[right_table]].oned_bin_modes
        key_group_pdf = dict()
        key_group_bin_mode = dict()
        new_union_key_group = dict()
        res = cond_factor_right.table_len
        new_na_values = dict()
        #print(equivalent_key_group)
        for key_group in equivalent_key_group:
            #print(key_group)
            #print("========================")
            #print(bin_mode_left.keys())
            #print(equivalent_key_group[key_group][left_table])
            #print("==========================================")
            #print(bin_mode_right.keys())
            #print(equivalent_key_group[key_group][right_table])
            all_pdfs = [cond_factor_left.pdfs[key] * cond_factor_left.table_len * cond_factor_left.na_values[key]
                        for key in equivalent_key_group[key_group][left_table]] + \
                       [cond_factor_right.pdfs[key] * res * cond_factor_right.na_values[key]
                        for key in equivalent_key_group[key_group][right_table]]
            all_bin_modes = [bin_mode_left[key] for key in equivalent_key_group[key_group][left_table]] + \
                            [bin_mode_right[key] for key in equivalent_key_group[key_group][right_table]]
            #print("====================================================")
            #print(equivalent_key_group[key_group][left_table])
            #print(equivalent_key_group[key_group][right_table])
            new_pdf, new_bin_mode = self.compute_bound_oned(all_pdfs, all_bin_modes, return_factor=True)
            res = np.sum(new_pdf)
            key_group_pdf[key_group] = new_pdf / res
            key_group_bin_mode[key_group] = new_bin_mode
            new_union_key_group[key_group] = [key_group]
            new_na_values[key_group] = 1.0

        for group in union_key_group:
            table, keys = union_key_group[group]
            new_union_key_group[group] = keys
            for key in keys:
                key_group_pdf[key] = conditional_factors[table].pdfs[key]
                key_group_bin_mode[key] = self.table_buckets[tables_all[table]].oned_bin_modes[key]
                new_na_values[key] = conditional_factors[table].na_values[key]

        new_factor = Group_Factor(sub_plan_query_str, res, key_group_pdf, key_group_bin_mode,
                                  union_key_group_set, new_union_key_group, new_na_values, new_join_cond)
        return new_factor, res

    def get_join_keys_two_tables(self, left_table, right_table, table_equivalent_group, table_key_equivalent_group,
                                 table_key_group_map, join_cond, join_keys, tables_all):
        """
            Get the join keys between two tables
        """
        actual_join_cond = []
        for cond in join_cond[left_table]:
            if cond in join_cond[right_table]:
                actual_join_cond.append(cond)
        equivalent_key_group = dict()
        union_key_group_set = table_equivalent_group[left_table].union(table_equivalent_group[right_table])
        union_key_group = dict()
        new_join_cond = join_cond[left_table].union(join_cond[right_table])
        if len(actual_join_cond) != 0:
            for cond in actual_join_cond:
                key1 = cond.split("=")[0].strip()
                key2 = cond.split("=")[1].strip()
                if key1.split(".")[0] == left_table:
                    key_left = key1.replace(left_table, tables_all[left_table])
                    key_group = table_key_group_map[left_table][key_left]
                    if key_group not in equivalent_key_group:
                        equivalent_key_group[key_group] = dict()
                    if left_table in equivalent_key_group[key_group]:
                        equivalent_key_group[key_group][left_table].append(key_left)
                    else:
                        equivalent_key_group[key_group][left_table] = [key_left]
                    assert key2.split(".")[0] == right_table, f"unrecognized table alias"
                    key_right = key2.replace(right_table, tables_all[right_table])
                    key_group_t = table_key_group_map[right_table][key_right]
                    assert key_group_t == key_group, f"key group mismatch for join {cond}"
                    if right_table in equivalent_key_group[key_group]:
                        equivalent_key_group[key_group][right_table].append(key_right)
                    else:
                        equivalent_key_group[key_group][right_table] = [key_right]
                else:
                    assert key2.split(".")[0] == left_table, f"unrecognized table alias"
                    key_left = key2.replace(left_table, tables_all[left_table])
                    key_group = table_key_group_map[left_table][key_left]
                    if key_group not in equivalent_key_group:
                        equivalent_key_group[key_group] = dict()
                    if left_table in equivalent_key_group[key_group]:
                        equivalent_key_group[key_group][left_table].append(key_left)
                    else:
                        equivalent_key_group[key_group][left_table] = [key_left]
                    assert key1.split(".")[0] == right_table, f"unrecognized table alias"
                    key_right = key1.replace(right_table, tables_all[right_table])
                    key_group_t = table_key_group_map[right_table][key_right]
                    assert key_group_t == key_group, f"key group mismatch for join {cond}"
                    if right_table in equivalent_key_group[key_group]:
                        equivalent_key_group[key_group][right_table].append(key_right)
                    else:
                        equivalent_key_group[key_group][right_table] = [key_right]

            for group in union_key_group_set:
                if group in equivalent_key_group:
                    continue
                elif group in table_key_equivalent_group[left_table]:
                    union_key_group[group] = (left_table, table_key_equivalent_group[left_table][group])
                else:
                    union_key_group[group] = (right_table, table_key_equivalent_group[right_table][group])

        else:
            common_key_group = table_equivalent_group[left_table].intersection(table_equivalent_group[right_table])
            common_key_group = list(common_key_group)[0]
            for group in union_key_group_set:
                if group == common_key_group:
                    equivalent_key_group[group] = dict()
                    equivalent_key_group[group][left_table] = table_key_equivalent_group[left_table][group]
                    equivalent_key_group[group][right_table] = table_key_equivalent_group[right_table][group]
                elif group in table_key_equivalent_group[left_table]:
                    union_key_group[group] = (left_table, table_key_equivalent_group[left_table][group])
                else:
                    union_key_group[group] = (right_table, table_key_equivalent_group[right_table][group])

        return equivalent_key_group, union_key_group_set, union_key_group, new_join_cond

    def get_additional_joins_two_tables(self, left_table, right_table, table_equivalent_group,
                                        table_key_equivalent_group):
        common_key_group = table_equivalent_group[left_table].intersection(table_equivalent_group[right_table])
        union_key_group_set = table_equivalent_group[left_table].union(table_equivalent_group[right_table])
        union_key_group = dict()
        all_join_predicates = []
        for group in union_key_group_set:
            if group in common_key_group:
                left_key = table_key_equivalent_group[left_table][group][0]
                left_key = left_table + "." + left_key.split(".")[-1]
                right_key = table_key_equivalent_group[right_table][group][0]
                right_key = right_table + "." + right_key.split(".")[-1]
                join_predicate = left_key + " = " + right_key
                all_join_predicates.append(join_predicate)
            if group in table_key_equivalent_group[left_table]:
                left_key = table_key_equivalent_group[left_table][group][0]
                left_key = left_table + "." + left_key.split(".")[-1]
                union_key_group[group] = left_key
            else:
                right_key = table_key_equivalent_group[right_table][group][0]
                right_key = right_table + "." + right_key.split(".")[-1]
                union_key_group[group] = right_key

        return all_join_predicates, union_key_group

    def get_sub_plan_join_key(self, sub_plan_query, join_cond):
        # returning a subset of join_keys covered by the tables in sub_plan_query
        touched_join_cond = set()
        untouched_join_cond = set()
        for tab in join_cond:
            if tab in sub_plan_query:
                touched_join_cond = touched_join_cond.union(join_cond[tab])
            else:
                untouched_join_cond = untouched_join_cond.union(join_cond[tab])
        touched_join_cond -= untouched_join_cond

        join_keys = dict()
        for cond in touched_join_cond:
            key1 = cond.split("=")[0].strip()
            table1 = key1.split(".")[0].strip()
            if table1 not in join_keys:
                join_keys[table1] = set([key1])
            else:
                join_keys[table1].add(key1)

            key2 = cond.split("=")[1].strip()
            table2 = key2.split(".")[0].strip()
            if table2 not in join_keys:
                join_keys[table2] = set([key2])
            else:
                join_keys[table2].add(key2)

        return join_keys



In [17]:
import pickle
with open("ground_truth_factors_no_filter.pkl", "rb") as f:
    ground_truth_factors_no_filter = pickle.load(f)

In [18]:
import pickle5 as pickle
import numpy as np
import os

def load_sample_imdb(table_buckets, tables_alias, query_file_orders, join_keys, table_key_equivalent_group,
                     SPERCENTAGE=1.0, qdir="/home/ubuntu/data_CE/saved_models/binned_cards/{}/job/all_job/"):
    qdir = qdir.format(SPERCENTAGE)
    all_sample_factors = []
    for fn in query_file_orders:
        conditional_factors = load_sample_imdb_one_query(table_buckets, tables_alias, fn, join_keys,
                                                         table_key_equivalent_group, SPERCENTAGE, qdir)
        all_sample_factors.append(conditional_factors)
    return all_sample_factors


def load_sample_imdb_one_query(table_buckets, tables_alias, query_file_name, join_keys, table_key_equivalent_group,
                               SPERCENTAGE=10.0, qdir="/home/ubuntu/data_CE/saved_models/binned_cards/{}/job/all_job/"):
    qdir = qdir.format(SPERCENTAGE)
    fpath = os.path.join(qdir, query_file_name)
    with open(fpath, "rb") as f:
        data = pickle.load(f)

    conditional_factors = dict()
    table_pdfs = dict()
    filter_size = dict()
    for i, alias in enumerate(data["all_aliases"]):
        column = data["all_columns"][i]
        alias = alias[0]
        key = tables_alias[alias] + "." + column
        cards = data["results"][i][0]
        n_bins = table_buckets[tables_alias[alias]].bin_sizes[key]
        pdfs = np.zeros(n_bins)
        for (j, val) in cards:
            if j is None:
                j = 0
            pdfs[j] += val
        table_len = np.sum(pdfs)
        print(alias+"."+column, table_len, pdfs)
        if table_len == 0:
            # no sample satisfy the filter, set it with a small value
            #print("========================", alias+"."+column)
            table_len = 1
            pdfs = table_key_equivalent_group[tables_alias[alias]].pdfs[key]
        else:
            pdfs /= table_len
        if alias not in table_pdfs:
            table_pdfs[alias] = dict()
            filter_size[alias] = table_len
        table_pdfs[alias][key] = pdfs

    for alias in tables_alias:
        if alias in table_pdfs:
            table_len = min(table_key_equivalent_group[tables_alias[alias]].table_len,
                            filter_size[alias]/(SPERCENTAGE/100))
            na_values = table_key_equivalent_group[tables_alias[alias]].na_values
            conditional_factors[alias] = Factor(tables_alias[alias], table_len, list(table_pdfs[alias].keys()),
                                                table_pdfs[alias], na_values)
        else:
            #TODO: ground-truth distribution
            conditional_factors[alias] = table_key_equivalent_group[tables_alias[alias]]
    return conditional_factors

In [20]:
q_file = "29b.pkl"
idx = q_file_names.index(q_file)
BE = Bound_ensemble(table_buckets, schema, ground_truth_factors_no_filter)
temp = BE.get_cardinality_bound_all(queries[idx], sub_plan_queries_str_all[idx], 
                                    q_file_names[idx], True, all_true_card)
                                    #true_card[q_file.split(".pkl")[0]])

NameError: name 'true_card' is not defined

In [21]:
all_true_card = np.load("/home/ubuntu/CEB/all_true_card_job.npy")

In [22]:
BE = Bound_ensemble(table_buckets, schema, ground_truth_factors_no_filter)
res = dict()
for q_file_id in range(len(queries)):
    temp = BE.get_cardinality_bound_all(queries[q_file_id], sub_plan_queries_str_all[q_file_id], q_file_names[q_file_id])
    res[q_file_names[q_file_id].split(".pkl")[0]] = temp

it3.id 1.0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
cct1.id 1.0 [1. 0.]
an.person_id 89637.0 [5.3653e+04 1.3933e+04 7.7240e+03 4.3310e+03 2.6190e+03 1.7270e+03
 1.0960e+03 8.2200e+02 5.5600e+02 5.0900e+02 3.2600e+02 3.0000e+02
 2.2700e+02 1.7600e+02 3.0100e+02 2.2900e+02 1.7000e+02 1.7100e+02
 1.1700e+02 1.5100e+02 9.7000e+01 1.0100e+02 9.2000e+01 7.1000e+01
 5.5000e+01 3.8000e+01 3.8000e+01 7.0000e+00 0.0000e+00]
ci.person_id 83877.0 [3.2274e+04 1.2645e+04 8.2360e+03 4.3730e+03 3.4830e+03 3.3990e+03
 1.9380e+03 2.1630e+03 1.7320e+03 9.4600e+02 7.4400e+02 1.4620e+03
 7.4500e+02 4.5300e+02 1.3350e+03 1.4700e+03 8.4900e+02 4.8200e+02
 9.4600e+02 4.6700e+02 5.1300e+02 3.9900e+02 2.1400e+02 2.3400e+02
 1.6700e+02 1.1800e+02 5.4000e+01 5.0000e+00 2.0310e+03]
ci.role_id 83877.0 [5.7194e+04 2.6651e+04 0.0000e+00 0.00

KeyError: 'cast_info.person_id'

In [23]:
q_file_names_real = []
sub_query_len = []
for query_no in range(1, 34):
    for suffix in ['a', 'b', 'c', 'd', 'e', 'f', 'g']:
        file = f"{query_no}{suffix}.sql"
        if file in os.listdir(query_path):
            q_file_names_real.append(file.split(".sql")[0])
            sub_query_len.append(len(res[file.split(".sql")[0]]))
            
start_idx = 0
true_card = dict()
all_true_card_new = []
for i, q_file in enumerate(q_file_names_real):
    print("=============================================")
    print(f"query no {i}, {q_file} with {len(res[q_file])} number of sub-plan queries")
    pred = res[q_file]
    pred = np.asarray(pred)
    end_idx = start_idx + sub_query_len[i]
    true = all_true_card[start_idx: end_idx]
    temp_true = copy.deepcopy(true)
    temp_true[true == -1] = pred[true == -1]
    all_true_card_new.append(temp_true)
    true_card[q_file] = true
    assert len(pred) == len(true)
    pred = pred[true != -1]
    pred[pred <= 0] = 1
    true = true[true != -1]
    q_errors = np.maximum(pred/true, true/pred)
    for i in [50, 90, 95, 99, 100]:
        print(f"q-error {i}% percentile is {np.percentile(q_errors, i)}")
    start_idx = end_idx
all_true_card_new = np.concatenate(all_true_card_new)

KeyError: '1a'

In [ ]:
ok = 0
with open("CE_scheme_est.txt", "w") as f:
    for t in q_file_names_real:
        for val in res[t]:
            ok += 1
            f.write(str(val)+"\n")

In [ ]:
with open("true_est_job.txt", "w") as f:
    for i in all_true_card_new:
        f.write(str(i)+"\n")

In [ ]:
ground_truth_factors_no_filter["char_name"].pdfs['char_name.id'] * ground_truth_factors_no_filter["char_name"].table_len

In [ ]:
temp[-1]

In [ ]:
model_path = "/Users/ziniuw/Desktop/research/Learned_QO/CC_model/CE_scheme_models/stats/model_200.pkl"
with open(model_path, "rb") as f:
    new_BE = pickle.load(f)

In [ ]:
query_str = "SELECT COUNT(*) FROM users as u, comments as c, posts as p WHERE p.OwnerUserId = u.Id AND p.Id = c.PostId AND u.UpVotes>=0 AND u.CreationDate>='2010-08-21 21:27:38'::timestamp AND c.CreationDate>='2010-07-21 11:05:37'::timestamp AND c.CreationDate<='2014-08-25 17:59:25'::timestamp"
t = time.time()
res = new_BE.get_cardinality_bound(query57)
print(time.time() - t)
print(res)

In [ ]:
query_file = "/Users/ziniuw/Desktop/past_research/End-to-End-CardEst-Benchmark/workloads/stats_CEB/sub_plan_queries/stats_CEB_sub_queries.sql"
with open(query_file, "r") as f:
    queries = f.readlines()


In [ ]:
qerror = []
latency = []
pred = []
for i, query_str in enumerate(queries):
    query = query_str.split("||")[0][:-1]
    print("========================")
    true_card = int(query_str.split("||")[-1])
    t = time.time()
    res = new_BE.get_cardinality_bound(query)
    pred.append(res)
    latency.append(time.time() - t)
    qerror.append(res/true_card)
    print(f"estimating query {i}: predicted {res}, true_card {true_card}, qerror {res/true_card}, latency {time.time() - t}")

In [ ]:
qerror = np.asarray(qerror)

In [ ]:
temp_qerror = copy.deepcopy(qerror)
temp_qerror[temp_qerror < 1] = 1/temp_qerror[temp_qerror < 1]

In [ ]:
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(temp_qerror, i)}")

In [ ]:
with open("stats_CEB_sub_queries_CE_scheme.txt", "w") as f:
    for p in pred:
        f.write(str(p)+"\n")

In [ ]:
with open("stats_CEB_exec.sql", "r") as f:
    queries = f.readlines()

In [ ]:
with open("stats_CEB_exec.sql", "w") as f:
    for q in queries:
        q = q.split("||")[-1]
        f.write(q)